In [ ]:
#DCGAN
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
from IPython.display import Image
# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

# Root directory for dataset
dataroot = "../input/celeba-dataset/img_align_celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 10

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

image_size = 64
DATA_DIR = "../input/pneumoniamnist/pneumoniamnist.npz"
X_train = np.load(DATA_DIR)
#print(f"Shape of training data: {X_train.shape}")
print(f"Data type: {type(X_train)}")
data = X_train["train_images"].astype(np.float64)
x = []
for img in data:
    x.append(np.dstack([img,img,img]))
x = np.array(x)
y = np.array(X_train["train_labels"])
data = x
data = 255 * data
img = data.astype(np.uint8)
X_train = img

class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)
    
transform = transforms.Compose([transforms.ToPILImage(),
                                transforms.Resize(64),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])
cropped_dataset = MyDataset(X_train, y, transform=transform)
print(len(cropped_dataset))

print(cropped_dataset[0][0].size())

# We can use an image folder dataset the way we have it setup.
# Create the dataset
# dataset = dset.ImageFolder(root=dataroot,
#                            transform=transforms.Compose([
#                                transforms.Resize(image_size),
#                                transforms.CenterCrop(image_size),
#                                transforms.ToTensor(),
#                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#                            ]))
# print(len(dataset))

# print(type(dataset))
# print(dataset[0][0].shape)

#dataset_subset = torch.utils.data.Subset(dataset, np.random.choice(len(dataset), 10000, replace=False))
#print(len(dataset_subset))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(cropped_dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers, pin_memory=True)

#dataloader1 = torch.utils.data.DataLoader(dataset_subset, batch_size = 64,shuffle=True, num_workers=workers)
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


real_batch = next(iter(dataloader))
#real_batch1 = next(iter(dataloader1))
print(real_batch[0].size())
#print(real_batch1[0].size())
plt.figure(figsize=(10,10))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device), padding=2, normalize=True,nrow=10,ncol=10).cpu(),(1,2,0)))

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.02.
netG.apply(weights_init)

# Print the model
print(netG)

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)

# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

from torch.nn.functional import adaptive_avg_pool2d
import torchvision.models as models
import torch.nn.functional as F
from scipy import linalg

class InceptionV3(nn.Module):
    """Pretrained InceptionV3 network returning feature maps"""

    # Index of default block of inception to return,
    # corresponds to output of final average pooling
    DEFAULT_BLOCK_INDEX = 3

    # Maps feature dimensionality to their output blocks indices
    BLOCK_INDEX_BY_DIM = {
        64: 0,   # First max pooling features
        192: 1,  # Second max pooling featurs
        768: 2,  # Pre-aux classifier features
        2048: 3  # Final average pooling features
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=True,
                 requires_grad=False):
        
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)

        assert self.last_needed_block <= 3, \
            'Last possible output block index is 3'

        self.blocks = nn.ModuleList()

        
        inception = models.inception_v3(pretrained=True)

        # Block 0: input to maxpool1
        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        # Block 1: maxpool1 to maxpool2
        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))

        # Block 2: maxpool2 to aux classifier
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        # Block 3: aux classifier to final avgpool
        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        for param in self.parameters():
            param.requires_grad = requires_grad

    def forward(self, inp):
        """Get Inception feature maps
        Parameters
        ----------
        inp : torch.autograd.Variable
            Input tensor of shape Bx3xHxW. Values are expected to be in
            range (0, 1)
        Returns
        -------
        List of torch.autograd.Variable, corresponding to the selected output
        block, sorted ascending by index
        """
        outp = []
        x = inp

        if self.resize_input:
            x = F.interpolate(x,
                              size=(299, 299),
                              mode='bilinear',
                              align_corners=False)

        if self.normalize_input:
            x = 2 * x - 1  # Scale from range (0, 1) to range (-1, 1)

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        return outp
    
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
model = InceptionV3([block_idx])
model=model.cuda()

def calculate_activation_statistics(images,model,batch_size=128, dims=2048,
                    cuda=False):
    model.eval()
    act=np.empty((len(images), dims))
    
    if cuda:
        batch=images.cuda()
    else:
        batch=images
    pred = model(batch)[0]

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.
    if pred.size(2) != 1 or pred.size(3) != 1:
        pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

    act= pred.cpu().data.numpy().reshape(pred.size(0), -1)
    
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma

def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

def calculate_fretchet(images_real,images_fake,model):
     mu_1,std_1=calculate_activation_statistics(images_real,model,cuda=True)
     mu_2,std_2=calculate_activation_statistics(images_fake,model,cuda=True)
    
     """get fretched distance"""
     fid_value = calculate_frechet_distance(mu_1, std_1, mu_2, std_2)
     return fid_value

print("Generator Parameters:",sum(p.numel() for p in netG.parameters() if p.requires_grad))
print("Discriminator Parameters:",sum(p.numel() for p in netD.parameters() if p.requires_grad))

# Commented out IPython magic to ensure Python compatibility.
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
FID = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
#                   % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        fretchet_dist=calculate_fretchet(real_cpu,fake,model)
        FID.append(fretchet_dist)
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 100 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
plt.title("FID Score During Training")
plt.plot(FID,label="FID")
plt.xlabel("iterations")
plt.ylabel("score")
plt.legend()
plt.show()

#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

In [ ]:
#VAE
VAE for PneumoniaMNINST dataset


from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as K
from sklearn.model_selection import train_test_split
from skimage.transform import resize

epochs=150
batch_size=8
input_shape=(28,28,1)
latent_dim=2
h_image,w_image=28,28
path = '/content/drive/MyDrive/PRNN/pneumoniamnist.npz'

def process_data(path):
    dataset = np.load(path)
    train_x, train_y = dataset['train_images'], dataset['train_labels']
    val_x, val_y = dataset['val_images'], dataset['val_labels']
    test_x, test_y = dataset['test_images'], dataset['test_labels']
    return train_x, train_y, test_x, test_y, val_x, val_y

#(X_train,y_train) , (X_test,y_test) = tf.keras.datasets.mnist.load_data()
X_train, y_train, X_test, y_test, t, d = process_data(path)
X_train = X_train.reshape(X_train.shape[0],28,28,1).astype('float32')/255.0
X_test = X_test.reshape(X_test.shape[0],28,28,1).astype('float32')/255.0
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

train_dataset=tf.data.Dataset.from_tensor_slices((X_train,y_train)).\
batch(batch_size).repeat()

test_dataset=tf.data.Dataset.from_tensor_slices((X_test,y_test)).\
batch(batch_size).repeat()

import time

def reparameterize(mean, logvar):
    eps = tf.random.normal(shape=tf.shape(mean))
    return eps * tf.exp(logvar * .5) + mean  

def model(input_shape,latent_dim):
    #Encoder 
    model_input = tf.keras.Input(input_shape)
    layer = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides = 1, activation = 'relu', padding = 'same')(model_input)
    layer = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, strides = 2, activation = 'relu', padding = 'same')(layer)
    layer = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides = 2, activation = 'relu', padding = 'same')(layer)
    shape_before_flatten=layer.shape
    layer = tf.keras.layers.Flatten()(layer)
    mean = tf.keras.layers.Dense(latent_dim)(layer)
    logvar = tf.keras.layers.Dense(latent_dim)(layer)
    encoder_model = tf.keras.models.Model(model_input, [mean, logvar])
  
    #Decoder
    decoder_input = tf.keras.Input((latent_dim,))
    layer = tf.keras.layers.Dense(np.prod(shape_before_flatten[1:]),\
                              activation = 'relu')(decoder_input)
    layer = tf.keras.layers.Reshape(target_shape=shape_before_flatten[1:])(layer)
    layer = tf.keras.layers.Conv2DTranspose(filters = 32, kernel_size = 3, strides = 2, activation = 'relu', padding = 'same')(layer)
    layer = tf.keras.layers.Conv2DTranspose(filters = 16, kernel_size = 3, strides = 2, activation = 'relu', padding = 'same')(layer)
    layer = tf.keras.layers.Conv2DTranspose(filters = 1, kernel_size = 3, strides = 1, activation = 'sigmoid', padding = 'same')(layer)
    decoder_model=tf.keras.models.Model(decoder_input, layer)  
 
    #Reparameterization Trick
    mean, logvar = encoder_model(model_input)
    z = reparameterize(mean, logvar)
    model_out = decoder_model(z)
    model = tf.keras.models.Model(model_input,model_out)

    #Reconstruction loss
    reconstruction_loss = K.sum(K.binary_crossentropy(model_input,
                                                    model_out), axis = [1, 2, 3])
    #KL div loss
    kl_loss = - 0.5 * K.sum(1 + logvar - K.square(mean) - K.exp(0.5 * logvar), axis = -1)
    elbo = K.mean(reconstruction_loss + kl_loss)
    
    model.add_loss(elbo)
    return model, decoder_model, encoder_model

filepath = '/content/drive/MyDrive/PRNN/checkpoint/weights_{epoch:02d}.hdf5'
vae_model,decoder_model,encoder_model = model(input_shape,latent_dim)
vae_model.compile(optimizer = tf.keras.optimizers.Adam(5e-3))
callback1 = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 10)
callback2 = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=0, save_freq=588)
history = vae_model.fit(train_dataset, epochs = epochs, callbacks = [callback1, callback2], steps_per_epoch = (X_train.shape[0]//batch_size))

#Plot the model training loss
loss_vae = history.history['loss']
plt.plot(loss_vae)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("VAE Training loss")
plt.grid()
plt.show()

#Testing using decoder
img_size=(h_image,w_image)
rows = 8
cols = 8

fig1, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(5,5))
for idx in range(rows*cols):
    image = X_test[idx, :, :,0]
    row = idx // cols
    col = idx % cols
    axes[row, col].axis("off")
    axes[row, col].imshow(image, cmap="gray", aspect="auto")
plt.subplots_adjust(wspace=.01, hspace=.01)
plt.show()

print("\n\n\n\n")

fig2, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(5,5))
for idx in range(rows*cols):
    mean, logvar = encoder_model.predict(np.expand_dims(X_test[idx, :, :, 0],axis=0))
    z = reparameterize(mean, logvar)
    output = decoder_model.predict(z)
    image = tf.reshape(output,img_size)
    row = idx // cols
    col = idx % cols
    axes[row, col].axis("off")
    axes[row, col].imshow(image, cmap="gray", aspect="auto")
plt.subplots_adjust(wspace=.01, hspace=.01)
plt.show()

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.nn.functional import adaptive_avg_pool2d
import torchvision.models as models
import torch.nn.functional as F
from scipy import linalg

class InceptionV3(nn.Module):
    """Pretrained InceptionV3 network returning feature maps"""

    # Index of default block of inception to return,
    # corresponds to output of final average pooling
    DEFAULT_BLOCK_INDEX = 3

    # Maps feature dimensionality to their output blocks indices
    BLOCK_INDEX_BY_DIM = {
        64: 0,   # First max pooling features
        192: 1,  # Second max pooling featurs
        768: 2,  # Pre-aux classifier features
        2048: 3  # Final average pooling features
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=True,
                 requires_grad=False):
        
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)

        assert self.last_needed_block <= 3, \
            'Last possible output block index is 3'

        self.blocks = nn.ModuleList()

        
        inception = models.inception_v3(pretrained=True)

        # Block 0: input to maxpool1
        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        # Block 1: maxpool1 to maxpool2
        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))

        # Block 2: maxpool2 to aux classifier
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        # Block 3: aux classifier to final avgpool
        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        for param in self.parameters():
            param.requires_grad = requires_grad

    def forward(self, inp):
        """Get Inception feature maps
        Parameters
        ----------
        inp : torch.autograd.Variable
            Input tensor of shape Bx3xHxW. Values are expected to be in
            range (0, 1)
        Returns
        -------
        List of torch.autograd.Variable, corresponding to the selected output
        block, sorted ascending by index
        """
        outp = []
        x = inp

        if self.resize_input:
            x = F.interpolate(x,
                              size=(299, 299),
                              mode='bilinear',
                              align_corners=False)

        if self.normalize_input:
            x = 2 * x - 1  # Scale from range (0, 1) to range (-1, 1)

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        return outp
    
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
fid_model = InceptionV3([block_idx])
fid_model = fid_model.cuda()

def calculate_activation_statistics(images,model,batch_size=128, dims=2048,
                    cuda=False):
    model.eval()
    act=np.empty((len(images), dims))
    
    if cuda:
        batch=images.cuda()
    else:
        batch=images
    pred = model(batch)[0]

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.
    if pred.size(2) != 1 or pred.size(3) != 1:
        pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

    act= pred.cpu().data.numpy().reshape(pred.size(0), -1)
    
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma

def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

def calculate_fretchet(images_real,images_fake,model):
     mu_1,std_1=calculate_activation_statistics(images_real,model,cuda=True)
     mu_2,std_2=calculate_activation_statistics(images_fake,model,cuda=True)
    
     """get fretched distance"""
     fid_value = calculate_frechet_distance(mu_1, std_1, mu_2, std_2)
     return fid_value

block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
model = InceptionV3([block_idx])
model = model.cuda()
avg_fid_score = 0.0
img_size=(h_image,w_image)

for i in range(10):
    mean, logvar = encoder_model.predict(np.expand_dims(X_test[i, :, :, 0],axis=0))
    z = reparameterize(mean, logvar)
    output = decoder_model.predict(z)
    output = tf.reshape(output,img_size)
    c = resize(np.asarray(X_test[i, :, :, 0]), (64, 64), anti_aliasing=True)
    d = resize(np.asarray(output), (64, 64), anti_aliasing=True)
    e = np.zeros((3, 64, 64))
    for i in range(3):
        e[i, :, :] = c
    f = np.zeros((3, 64, 64))
    for i in range(3):
        f[i, :, :] = d
    a = torch.from_numpy(e)
    b = torch.from_numpy(f)
    a = torch.from_numpy(np.expand_dims(a, 0)).float()
    b = torch.from_numpy(np.expand_dims(b, 0)).float()
    avg_fid_score += calculate_fretchet(a, b, model)
print(avg_fid_score/10.0)

In [ ]:
#PCA
"""PCA for PneumoniaMNIST dataset"""

import numpy as np
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap

def process_data(path):
    dataset = np.load(path)
    train_x, train_y = dataset['train_images'], dataset['train_labels']
    val_x, val_y = dataset['val_images'], dataset['val_labels']
    test_x, test_y = dataset['test_images'], dataset['test_labels']
    return train_x, train_y, test_x, test_y

def pca(X_train, num_components):

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]**2))

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)

    pca = PCA(n_components=num_components)
    x_ = pca.fit_transform(X_train_scaled)
    return x_

def plot_PCA(x_, y_train, plot_points, num_components):
    if(num_components == 3):
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        sc = ax.scatter(x_[0:plot_points, 0], x_[0:plot_points, 1], x_[0:plot_points, 2], c=y_train[0:plot_points], marker='o', cmap='coolwarm', alpha=1)
        plt.legend(*sc.legend_elements(), bbox_to_anchor=(1.05, 1), loc=2)
        plt.savefig("3d.png")
        plt.title("PCA with 3 principal components")
        ax.set_xlabel("PC1")
        ax.set_ylabel("PC2")
        ax.set_zlabel("PC3")
    else:
        sc = plt.scatter(x_[0:plot_points, 0], x_[0:plot_points, 1], c=y_train[0:plot_points], marker='o', cmap='coolwarm', alpha=1)
        plt.legend(*sc.legend_elements(), bbox_to_anchor=(1.05, 1), loc=2)
        plt.savefig("2d.png")
        plt.title("PCA with 2 principal components") 
        plt.xlabel("PC1")
        plt.ylabel("PC2") 
    plt.grid()      
    plt.show()
    return

def main():
    path = 'drive/MyDrive/PRNN/pneumoniamnist.npz'
    X_train, y_train, test_x, test_y = process_data(path)
    n = 3
    x = pca(X_train, num_components=n)
    plot_PCA(x, y_train, plot_points=60000, num_components=n)
    return

if __name__=='__main__':
    main()

In [ ]:
#tsne for pneumoniamnist

data = np.load("../input/pneumoniamnist/pneumoniamnist.npz" )
train_x = data["train_images"]
train_y = data["train_labels"]
X = np.reshape(train_x,(4708,784))/255.0
y = np.reshape(train_y,(4708))

print(X.shape)
print(y.shape)

# Commented out IPython magic to ensure Python compatibility.

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

feat_cols = [ 'pixel'+str(i) for i in range(X.shape[1]) ]
df = pd.DataFrame(X,columns=feat_cols)
df['y'] = y
df['label'] = df['y'].apply(lambda i: str(i))
X, y = None, None
print('Size of the dataframe: {}'.format(df.shape))

# For reproducability of the results
np.random.seed(42)
rndperm = np.random.permutation(df.shape[0])

plt.gray()
fig = plt.figure( figsize=(16,7) )
for i in range(0,15):
    ax = fig.add_subplot(3,5,i+1, title="Digit: {}".format(str(df.loc[rndperm[i],'label'])) )
    ax.matshow(df.loc[rndperm[i],feat_cols].values.reshape((28,28)).astype(float))
plt.show()

N = 4708
df_subset = df.loc[rndperm[:N],:].copy()
data_subset = df_subset[feat_cols].values
#print(df_subset)
print(df_subset['label'].shape)
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(data_subset)

df_subset['tsne-2d-one'] = tsne_results[:,0]
df_subset['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(10,6))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 2),
    data=df_subset,
    legend="full",
    alpha=0.3
)

import plotly.express as px
labels = df_subset["label"]

df0 = pd.DataFrame({
'label': np.squeeze(labels), 'col_x':tsne_results[:,0], 'col_y':tsne_results[:,1]
})

fig = px.scatter(df0, x='col_x', y='col_y', color='label',
                 width=700, height=500,
                 title="2D Scatter Plot")
fig.show()

tsne1 = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
tsne1_results = tsne1.fit_transform(data_subset)
print(tsne_results.shape)

labels = df_subset["label"]
print(np.squeeze(tsne1_results[:,0]).shape)
print(np.squeeze(labels).shape)
df1 =  pd.DataFrame({
'label': np.squeeze(labels), 'col_x':tsne1_results[:,0], 'col_y':tsne1_results[:,1], 'col_z':tsne1_results[:,2]
})

fig = px.scatter_3d(df1, x='col_x', y='col_y', z='col_z',
                    color='label',
                    title="3D Scatter Plot")
fig.show()

In [ ]:
#PCA reconstruction
#in this code we will use PCA to reduce the dimension of the data
# and then we will use the reduced data to reconstruct the original data
#this is an experiment to see how much reconstruction error we can get
# we will pneumoniamnist.npy dataset


#Global variable

# path = 'drive/MyDrive/Colab Notebooks/PRNN_A1_DATA/'
import os

from sklearn.preprocessing import StandardScaler

path = os.getcwd() + '/'

#importing Libraries
import numpy as np
from matplotlib.image import imread
import matplotlib.pyplot as plt
# import cv2
import scipy.linalg as la

from sklearn.decomposition import PCA, IncrementalPCA

#pre-processing the data


def data_preprocessing():
    data_path = path + 'pneumoniamnist.npz'
    data = np.load(data_path)
    # print(data.files)
    # Reshape images and save in numpy arrays
    train1_data = np.reshape(data['train_images'], (4708, 784))


    train1_data = train1_data.astype(float)
    data_vectors = train1_data

    return data_vectors

#form the data covariance matrix

def data_cov_matrix(data_vectors):
    data_covariance_matrix = np.cov(data_vectors.T)

    #plot all the eigenvalues largest to smallest
    eigenvalues, eigenvectors = la.eig(data_covariance_matrix)

    #print number of eigenvalues
    print('Number of eigenvalues: ', len(eigenvalues))
    # #plot eigen values on scale of 10^(5)
    # temp = eigenvalues*(10**(-5))
    # plt.plot(temp)
    # plt.title('Eigenvalues of Data Covariance matrix' )
    # #y axis label
    # plt.ylabel('Eigenvalues x 10^5')
    # plt.show()

    #plottying residual error
    #making a vector J same length as eigenvalues and is generated as
    # J_i = sum(eigenvalues[i+1:])
    J = np.zeros(len(eigenvalues))
    for i in range(len(eigenvalues)):
        J[i] = sum(eigenvalues[i+1:])

    #plotting eigen values and residual error in same plot
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    temp_eigenvalues = eigenvalues*(10**(-5))
    plt.plot(temp_eigenvalues)
    plt.title('Eigenvalues of Data Covariance matrix')
    plt.ylabel('Eigenvalues x 10^5')
    #x axis label i
    plt.xlabel('i')
    plt.subplot(1,2,2)
    temp_J = J*(10**(-5))
    plt.plot(temp_J)
    plt.title('Residual Error ')
    plt.ylabel('Residual Error x 10^5')
    plt.xlabel('M')
    plt.show()






    return data_covariance_matrix


#plotting eigenvectors as images

def eigen_vectors_images(data_covariance_matrix):
    eigenvalues, eigenvectors = la.eig(data_covariance_matrix)
    #print number of eigenvalues
    # print('Number of eigenvalues: ', len(eigenvalues))
    #plot eigen values on scale of 10^(5)
    # temp = eigenvalues*(10**(-5))
    # plt.plot(temp)
    # plt.title('Eigenvalues of Data Covariance matrix' )
    # #y axis label
    # plt.ylabel('Eigenvalues x 10^5')
    # plt.show()

    #plotting eigen vectors as images
    #we will plot the first 10 eigen vectors in same plot
    plt.figure(figsize=(10,10))
    for i in range(10):
        plt.subplot(2,5,i+1)
        plt.imshow(eigenvectors[:,i].reshape(28,28), cmap='gray')
        plt.title('Eigenvector ' + str(i+1))
        plt.axis('off')
    plt.tight_layout()
    plt.show()


# def PCA_reconstruction(input_vector, k, ):
#
#     #here input vector is 1 single vector of data set
# #k is the number of eigenvectors we want to use for first PCA step
#     #we will use the top k eigenvectors to reconstruct the input vector
#
#     #getting the t


def return_U_matrix(eigenvectors, k):
    #here eigenvectors is the matrix of eigenvectors
    #k is the number of eigenvectors we want to use for first PCA step
    #we will use the top k eigenvectors to reconstruct the input vector
    U = eigenvectors[:, :k]
    return U


def forward_PCA(input_vector, k, eigenvectors):
    #here input vector is 1 single vector of data set
    #k is the number of eigenvectors we want to use for first PCA step
    #making U^T
    U_transpose = return_U_matrix(eigenvectors, k).T
    y = U_transpose.dot(input_vector)
    return y

def reconstruct_PCA(y, k, eigenvectors):
    #y is a single output vector
#k is the number of eigenvectors we want to use for first PCA step
    #making U
    U = return_U_matrix(eigenvectors, k)
    x = U.dot(y)
    return x


def reconstruction(k, image_vector, eigenvectors):
    #forward PCA
    y = forward_PCA(image_vector, k, eigenvectors)
    #reconstruction
    x = reconstruct_PCA(y, k, eigenvectors)
    return x




#doing PCA on the data
def PCA(data_vectors):

    #mean centre the data
    mean_centre_data = data_vectors - np.mean(data_vectors, axis=0)
    data_vectors = mean_centre_data
    #get the covariance matrix
    data_covariance_matrix = np.cov(data_vectors.T)
    #get the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = la.eig(data_covariance_matrix)
    #sort the eigenvalues and eigenvectors in descending order
    eigenvalues = np.sort(eigenvalues)[::-1]
    eigenvectors = eigenvectors[:, eigenvalues.argsort()[::-1]]

    #print eigenvalues
    # print('Eigenvalues of Data Covariance matrix: ', eigenvalues)
    #print number of eigenvalues
    # print('Number of eigenvalues: ', len(eigenvalues))
    #print eigrnvectors

    # print('Eigenvectors of Data Covariance matrix: ', eigenvectors)
    #print first eigenvector
    #print shape of eigenvectors
    # print('Shape of eigenvectors: ', eigenvectors.shape)
    # print('First Eigenvector of Data Covariance matrix: ', eigenvectors[:,0])

    #original image
    image_vector = data_vectors[0]

    k_values = [1 , 2 , 10,  50, 100, 500, 784]

    #getting the reconstructed images for all k
    reconstructed_images = []
    for k in k_values:
        reconstructed_images.append(reconstruction(k, image_vector, eigenvectors))

    #calculating MSE for all k
    MSE_values = []
    for i in range(len(reconstructed_images)):
        MSE_values.append(np.mean((reconstructed_images[i] - image_vector)**2))

    # calculating the reconstruction error for all k
    reconstruction_error = []
    for i in range(len(reconstructed_images)):
        #reconstruction error is the sum of eigenvalues from k+1 to 784
        reconstruction_error.append(np.sum(eigenvalues[k_values[i]+1:]))

    #plotting the reconstructed images and the original image
    plt.figure(figsize=(10,10))
    for i in range(len(reconstructed_images)+1):
        plt.subplot(1,8,i+1)
        #if i is 0 then plot the original image
        if i == 0:
            plt.imshow(image_vector.reshape(28,28), cmap='gray')
            plt.title('Original')
            plt.axis('off')
        #if i is not 0 then plot the reconstructed image
        else:
            plt.imshow(reconstructed_images[i-1].reshape(28,28), cmap='gray')
            plt.title('k=' + str(k_values[i-1]))
            plt.axis('off')
    plt.tight_layout()
    plt.show()

    #plotting MSE vs k
    plt.figure(figsize=(10,10))
    #plot line graph
    plt.plot(k_values, MSE_values)
    plt.xlabel('k')
    plt.ylabel('MSE')
    plt.title('MSE vs k')
    plt.show()

    #plotting reconstruction error vs k
    plt.figure(figsize=(10,10))
    #plot line graph
    plt.plot(k_values, reconstruction_error)
    plt.xlabel('k')
    plt.ylabel('Reconstruction Error')
    plt.title('Reconstruction Error vs k')
    plt.show()
















def main():
    data_vectors = data_preprocessing()
    # data_covariance_matrix = data_cov_matrix(data_vectors)
    # eigen_vectors_images(data_covariance_matrix)

    PCA(data_vectors)





if __name__ == '__main__':
    main()

In [ ]:
#KMeans
#this code will run kmeans clustering on the data and then use the kmeans clusters to find the nearest neighbors of the test data
#the cluster centers will be used as the representative points for the kmeans clusters
# we will test for various values of k and find the best k value


#import the packages
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
#metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
#Global variable


# path = 'drive/MyDrive/Colab Notebooks/PRNN_A1_DATA/'
import os

from sklearn.preprocessing import StandardScaler

path = os.getcwd() + '/'


#data preprocessing
def data_preprocessing():
    data_path = path + 'pneumoniamnist.npz'
    data = np.load(data_path)
    # print(data.files)
    # Reshape images and save in numpy arrays
    train1_data = np.reshape(data['train_images'], (4708, 784))
    train1_labels = np.reshape(data['train_labels'], (4708))
    #change data type to float of train1_data
    train1_data = train1_data.astype(float)
    #change datatype of train1_labels to int
    train1_labels = train1_labels.astype(int)

    #validation data
    valid1_data = np.reshape(data['val_images'], (524, 784))
    valid1_labels = np.reshape(data['val_labels'], (524))
    #change datatype of valid1_data to float
    valid1_data = valid1_data.astype(float)
    #change datatype of valid1_labels to int
    valid1_labels = valid1_labels.astype(int)


    test1_data = np.reshape(data['test_images'], (624, 784))
    test1_labels = np.reshape(data['test_labels'], (624))
    #change data type to float of test1_data
    test1_data = test1_data.astype(float)
    #change datatype of test1_labels to int
    test1_labels = test1_labels.astype(int)

    # feature scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(train1_data , train1_labels)
    X_test = sc.fit_transform(test1_data, test1_labels)
    X_valid = sc.fit_transform(valid1_data, valid1_labels)

    Y_train = train1_labels

    Y_test = test1_labels

    Y_valid = valid1_labels

    return X_train, Y_train, X_test, Y_test , X_valid, Y_valid

#kmeans clustering

def kmeans_clustering(data, k):
    #run kmeans clustering
    kmeans = KMeans(n_clusters=k, random_state=0).fit(data)
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_

    # #print kmeans
    # print(kmeans)
    # #print labels
    # print(labels)
    return kmeans, labels, centroids


def get_centroid_in_dataset(centroids, X_train):
    new_centroids = []
    # using NearestNeighbors to find closed point to centroid in dataset
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='brute').fit(X_train)
    distances, indices = nbrs.kneighbors(centroids)
    # loop over indices
    for i in range(len(indices)):
        new_centroids.append(X_train[indices[i][0]])

    return new_centroids







#finding the y_label of the centroids
def find_centroid_label(centroids, data , labels):
    # find the nearest point to each centroid


    centroids= get_centroid_in_dataset(centroids, data)
    centroid_labels = []
    for i in range(len(centroids)):
        #search the data for the centroid
        for j in range(len(data)):
            if np.array_equal(centroids[i], data[j]):
                centroid_labels.append(labels[j])


    #list to numpy array
    centroid_labels = np.array(centroid_labels)
    return centroid_labels


#testing find_centroid_label function
def test_find_centroid_label(X_train, Y_train):


    centroids = [X_train[0], X_train[1]]
    #nparray
    centroids = np.array(centroids)
    centroid_labels = find_centroid_label(centroids, X_train, Y_train)
    # print(centroid_labels)
    #printing the centroid labels


#generating classification report
def classification_report_generator(labels, predicted_labels):
    # calculating metrics
    # accuracy
    accuracy = accuracy_score(labels, predicted_labels)
    # print("Accuracy: ", accuracy)
    # confusion matrix
    confusion_matrix_metric = confusion_matrix(labels, predicted_labels)
    # print("Confusion Matrix: \n", confusion_matrix_metric)
    # # plotting confusion matrix
    # plt.matshow(confusion_matrix_metric)
    # plt.title('Confusion Matrix')
    # plt.colorbar()
    # plt.ylabel('True label')
    # plt.xlabel('Predicted label')
    # plt.show()
    # classification report
    classification_report_metric = classification_report(labels, predicted_labels)
    # print("Classification Report: \n", classification_report_metric)
    # f1 score
    f1_score_metric = f1_score(labels, predicted_labels, average='weighted', zero_division=True)
    # print("F1 Score: ", f1_score_metric)
    # plotting ROC curve
    # fpr, tpr, thresholds = roc_curve(labels, predicted_labels)
    roc_auc = roc_auc_score(labels, predicted_labels)
    # plt.title('Receiver Operating Characteristic')
    # plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
    # plt.legend(loc='lower right')
    # plt.plot([0, 1], [0, 1], 'r--')
    # plt.xlim([0, 1])
    # plt.ylim([0, 1])
    # plt.ylabel('True Positive Rate')
    # plt.xlabel('False Positive Rate')
    # plt.show()
    # # printing AUC score
    # print("AUC: ", roc_auc)

    # saving the metric in dictionary
    metrics = {'Accuracy': accuracy, 'Confusion Matrix': confusion_matrix_metric,
               'Classification Report': classification_report_metric, 'F1 Score': f1_score_metric, 'ROC Curve': roc_auc,
               'AUC': roc_auc}
    return metrics

# k nearest neighbour classifier
def k_nearest_neighbour_classifier(data, labels, test_data, test_labels,k):

    # print(data.shape)
    # print(labels.shape)
    data = np.array(data)
    labels = np.array(labels)


    #nearest neighbour classifier
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(data, labels)
    #predict the labels
    predicted_labels = neigh.predict(test_data)

    #generating classification report
    # print("k nearest neighbour classifier")
    #
    # print(predicted_labels)
    # print(test_labels)
    # print(np.array(predicted_labels).shape)
    # print(np.array(test_labels).shape)

    metrics = classification_report_generator(test_labels, predicted_labels)

    return predicted_labels, metrics




def kmeans_knn_classifier(dataset, y_data, test_data, y_test,k):
    #training phase
    #run kmeans clustering
    kmeans, labels, centroids = kmeans_clustering(dataset, k)
    #find the centroid labels
    centroid_labels = find_centroid_label(centroids, dataset , y_data)

    # print("in kmeans_knn_classifier")
    # print(centroid_labels)

    # creating augmented dataset using the centroid labels and centroids
    data_augmented = centroids
    labels_augmented = centroid_labels

    # using k nearest neighbour classifier on augmented dataset
    ret_val = k_nearest_neighbour_classifier(data_augmented, labels_augmented, test_data, y_test,k)


    predicted_labels = ret_val[0]
    metrics = ret_val[1]

    # print("kmeans_knn_classifier")
    # print(predicted_labels)
    # print(y_test)
    # print(np.array(predicted_labels).shape)
    # print(np.array(y_test).shape)


    metrics = classification_report_generator(y_test, predicted_labels)
    return predicted_labels , metrics






#comparing the performance of these 2 different classifiers
def compare_classifiers(dataset, labels, test_data, test_labels, k):

    # using kmeans knn classifier
    predicted_labels , metrics = kmeans_knn_classifier(dataset, labels, test_data, test_labels, k)


    # using k nearest neighbour classifier
    predicted_labels_knn, metrics_knn = k_nearest_neighbour_classifier(dataset, labels, test_data, test_labels,k)

    #printing the accuracy of the classifiers
    print("Accuracy of Kmeans KNN Classifier: ", metrics['Accuracy'])
    print("Accuracy of KNN Classifier: ", metrics_knn['Accuracy'])

    #f1 score of the classifiers
    print("F1 Score of Kmeans KNN Classifier: ", metrics['F1 Score'])
    print("F1 Score of KNN Classifier: ", metrics_knn['F1 Score'])

    #plotting the confusion matrix of kmeans knn classifier
    plt.matshow(metrics['Confusion Matrix'])
    plt.title('Confusion Matrix Kmeans KNN Classifier k = ' + str(k))
    plt.colorbar()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

    #plotting the confusion matrix
    plt.matshow(metrics_knn['Confusion Matrix'])
    plt.title('Confusion Matrix KNN Classifier k = ' + str(k))
    plt.colorbar()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

    # #plot shape of labels
    # print(np.array(predicted_labels).shape)
    # print(np.array(test_labels).shape)

    #plotting both ROCCurves on the same plot with different colors and labelled
    fpr_kmeans, tpr_kmeans, thresholds_kmeans = roc_curve(test_labels, predicted_labels)
    fpr_knn, tpr_knn, thresholds_knn = roc_curve(test_labels, predicted_labels_knn)
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr_kmeans, tpr_kmeans, 'b', label='Kmeans KNN AUC = %0.2f' % metrics['ROC Curve'])
    plt.plot(fpr_knn, tpr_knn, 'r', label='KNN AUC = %0.2f' % metrics_knn['ROC Curve'])
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

    # # plotting ROC curve of kmeans knn classifier
    # fpr_kmeans, tpr_kmeans, thresholds_kmeans = roc_curve(labels, predicted_labels)
    # plt.title('Receiver Operating Characteristic')
    # plt.plot(fpr_kmeans, tpr_kmeans, 'b', label='Kmeans KNN AUC = %0.2f' % metrics['ROC Curve'])
    # plt.legend(loc='lower right')
    # plt.plot([0, 1], [0, 1], 'r--')
    # plt.xlim([0, 1])
    # plt.ylim([0, 1])
    # plt.ylabel('True Positive Rate')
    # plt.xlabel('False Positive Rate')
    # plt.show()


    #printing the AUC score of the classifiers
    print("AUC of Kmeans KNN Classifier: ", metrics['AUC'])
    print("AUC of KNN Classifier: ", metrics_knn['AUC'])



# now we will vary k and plot curves for accuracy and AUC at each k
def vary_k_plot_curves(dataset, labels, test_data, test_labels):
    # k = [1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10, 11,  15,  20, 50, 100, 500, 1000]
    k = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 20, 50, 75]
    accuracy_kmeans_knn = []

    auc_kmeans_knn = []
    accuracy_knn = []
    auc_knn = []

    best_k_for_kmeans_knn = 1
    best_k_for_knn = 1
    temp_l1 , temp_m1 = kmeans_knn_classifier(dataset, labels, test_data, test_labels, best_k_for_kmeans_knn)
    best_accuracy_for_kmeans_knn = temp_m1['Accuracy']
    temp_l2 , temp_m2 = k_nearest_neighbour_classifier(dataset, labels, test_data, test_labels, best_k_for_knn)
    best_accuracy_for_knn = temp_m2['Accuracy']



    for i in k:
        predicted_labels , metrics = kmeans_knn_classifier(dataset, labels, test_data, test_labels, i)
        predicted_labels_knn, metrics_knn = k_nearest_neighbour_classifier(dataset, labels, test_data, test_labels,i)
        accuracy_kmeans_knn.append(metrics['Accuracy'])
        auc_kmeans_knn.append(metrics['AUC'])
        accuracy_knn.append(metrics_knn['Accuracy'])
        auc_knn.append(metrics_knn['AUC'])

        if metrics['Accuracy'] > best_accuracy_for_kmeans_knn:
            best_accuracy_for_kmeans_knn = metrics['Accuracy']
            best_k_for_kmeans_knn = i
        if metrics_knn['Accuracy'] > best_accuracy_for_knn:
            best_accuracy_for_knn = metrics_knn['Accuracy']
            best_k_for_knn = i


    plt.plot(k, accuracy_kmeans_knn, 'b', label='Kmeans KNN Accuracy')
    plt.plot(k, accuracy_knn, 'r', label='KNN Accuracy')
    plt.legend(loc='lower right')
    plt.xlabel('k')
    plt.ylabel('Accuracy')
    plt.show()

    plt.plot(k, auc_kmeans_knn, 'b', label='Kmeans KNN AUC')
    plt.plot(k, auc_knn, 'r', label='KNN AUC')
    plt.legend(loc='lower right')
    plt.xlabel('k')
    plt.ylabel('AUC')
    plt.show()

    return best_k_for_kmeans_knn, best_k_for_knn, best_accuracy_for_kmeans_knn, best_accuracy_for_knn















def main():

    X_train, Y_train, X_test, Y_test, X_valid, Y_valid = data_preprocessing()
    # test_find_centroid_label(X_train, Y_train)


    # kmeans_clustering(data_vectors, 3)
    # compare_classifiers(X_train, Y_train, X_test,Y_test, 11)
    best_k_for_kmeans_knn, best_k_for_knn, best_accuracy_for_kmeans_knn, best_accuracy_for_knn = vary_k_plot_curves(X_train, Y_train, X_test, Y_test)

    #print best k
    print("best_k_for_kmeans_knn", best_k_for_kmeans_knn)
    print("best_accuracy_for_kmeans_knn", best_accuracy_for_kmeans_knn)
    print("best_k_for_knn", best_k_for_knn)
    print("best_accuracy_for_knn", best_accuracy_for_knn)

    # plot
    compare_classifiers(X_train, Y_train, X_test, Y_test, best_k_for_kmeans_knn)
    compare_classifiers(X_train, Y_train, X_test, Y_test,best_k_for_knn)







if __name__ == "__main__":
    main()

In [ ]:
#Adaboost
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics

from sklearn import tree
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

dtc = tree.DecisionTreeClassifier()
svc = SVC(probability=True, kernel='linear')
nnc = MLPClassifier(hidden_layer_sizes=(20,10))

dtcA = AdaBoostClassifier(n_estimators=20, base_estimator=dtc, learning_rate=1)
svcA = AdaBoostClassifier(n_estimators=5, base_estimator=svc, learning_rate=1)
nncA = AdaBoostClassifier(n_estimators=20, base_estimator=nnc, learning_rate=1)

dPnuModel = dtc.fit(pnuTrainX, pnuTrainY)
dPnuTestR = dPnuModel.predict(pnuTestX).tolist()
dPnuMatrix = confMatrix(pnuTestY, dPnuTestR, pnuClasses)
print("Accuracy:", accuracy(dPnuMatrix), ", f1-score:", "and auc:", roc_auc_score(dAPnuTestR, pnuTestY)))

sPnuModel = svc.fit(pnuTrainX, pnuTrainY)
sPnuTestR = sPnuModel.predict(pnuTestX).tolist()
sPnuMatrix = confMatrix(pnuTestY, sPnuTestR, pnuClasses)
print("Accuracy:", accuracy(sPnuMatrix), ", f1-score:", "and auc:", roc_auc_score(dAPnuTestR, pnuTestY)))

nPnuModel = nnc.fit(pnuTrainX, pnuTrainY)
nPnuTestR = nPnuModel.predict(pnuTestX).tolist()
nPnuMatrix = confMatrix(pnuTestY, nPnuTestR, pnuClasses)
print("Accuracy:", accuracy(nPnuMatrix), ", f1-score:", "and auc:", roc_auc_score(dAPnuTestR, pnuTestY)))

dAPnuModel = dtcA.fit(pnuTrainX, pnuTrainY)
dAPnuTestR = dAPnuModel.predict(pnuTestX).tolist()
dAPnuMatrix = confMatrix(pnuTestY, dAPnuTestR, pnuClasses)
print("Accuracy:", accuracy(dAPnuMatrix), ", f1-score:", f1Score(dAPnuMatrix), "and auc:", roc_auc_score(dAPnuTestR, pnuTestY))

sAPnuModel = svcA.fit(pnuTrainX, pnuTrainY)
sAPnuTestR = sAPnuModel.predict(pnuTestX).tolist()
sAPnuMatrix = confMatrix(pnuTestY, sAPnuTestR, pnuClasses)
print("Accuracy:", accuracy(sAPnuMatrix), ", f1-score:", f1Score(sAPnuMatrix), "and auc:", roc_auc_score(sAPnuTestR, pnuTestY))



